# Interactive Notebook of Nomisma Findspots

From the "Cell" menu near the top of this page choose, "Run All". Scroll down to the bottom of the page and you'll see a dropdown menu that will let you choose findspots; doing so will in turn display links to coins found at the findspot you've chosen.

In [ ]:
from ipywidgets import widgets,interact
from IPython.display import display, HTML, clear_output
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.request import urlopen
from rdflib import Graph, plugin

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', -1) # so that full Pleiades URIs are displayed

In [ ]:
sparql = SPARQLWrapper("http://nomisma.org/query")
sparql.setQuery("""

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo:	<http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nmo:	<http://nomisma.org/ontology#>

SELECT DISTINCT ?name WHERE {
   
   ?loc a geo:SpatialThing ;
        foaf:name ?name .
   
   ?id nmo:hasFindspot ?loc ;
       a nmo:NumismaticObject ;
       nmo:hasCollection ?collection .
       
} ORDER BY ?name""")
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

cols = results['head']['vars']

out = []
for row in results['results']['bindings']:
    item = []
    for c in cols:
        item.append(row.get(c, {}).get('value'))
    out.append(item)

nomisma_df = pd.DataFrame(out, columns=cols)
type(nomisma_df.name[0])

In [ ]:
def on_value_change(change=1): # change isn't used but will be passewd so there has to be a placeholder
    global dropdown
    
    try:
        chosen = dropdown.value
    except NameError:
        chosen = nomisma_df.name[0]
        
    sparql = SPARQLWrapper("http://nomisma.org/query")
    sparql.setQuery("""
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX nmo:	<http://nomisma.org/ontology#>

SELECT DISTINCT ?url ?collection ?obvimage ?revimage WHERE {
   ?url nmo:hasFindspot ?loc ;
   a nmo:NumismaticObject ;
   nmo:hasCollection ?collection .
   
   ?url nmo:hasObverse/foaf:thumbnail ?obvimage .
   ?url nmo:hasReverse/foaf:thumbnail ?revimage .
   
   ?loc foaf:name '%s' .
       
}""" % (chosen))
    
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()

    cols = results['head']['vars']

    out = []
    for row in results['results']['bindings']:
        item = []
        for c in cols:
            if c in ['obvimage','revimage']:
                item.append('<img src="{0}"></img>'.format(row.get(c, {}).get('value')))
            else:
                item.append('<a target="_new" href="{0}">{0}</a>'.format(row.get(c, {}).get('value')))
        out.append(item)
    
    items_df = pd.DataFrame(out, columns=cols)
    
    dropdown = widgets.Dropdown(
    options=nomisma_df['name'].tolist(),
    value = chosen,
    description='Findspot:',
    disabled=False)
    
    clear_output()
    dropdown.observe(on_value_change, names='value')
    
    display(dropdown, HTML(items_df.to_html(escape=False)))

on_value_change(change=1)